In [4]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import requests
url = 'https://biendata.com/competition/airquality/bj/2018-03-01-0/2018-04-28-23/2k0d1d8'
respones= requests.get(url)
with open ("bj_airquality_2018-03-01-0-2018-05-04-23.csv",'w') as f:
    f.write(respones.text)

In [5]:
air = pd.read_csv("bj_airquality_2018-03-01-0-2018-05-04-23.csv")

In [7]:
def func(station):
    return station.split('_')[0]
air['station_id'] = list(map(func, air['station_id'])) 

In [8]:
air.columns

Index(['id', 'station_id', 'time', 'PM25_Concentration', 'PM10_Concentration',
       'NO2_Concentration', 'CO_Concentration', 'O3_Concentration',
       'SO2_Concentration'],
      dtype='object')

In [9]:
air.columns = ['id', 'station_id', 'time', 'PM2.5', 'PM10', 'NO2', 'CO', 'O3','SO2']

In [10]:
air = air[['station_id', 'time', 'PM2.5', 'PM10', 'NO2', 'CO', 'O3','SO2']]

In [11]:
data = air[air['station_id'] == 'dongsi']

In [23]:
len(data)-48

591

In [27]:
act = data['PM2.5'][591:]

In [28]:
data = data[:591]

In [29]:
def feature_vector(df, feature, N):
    rows = df.shape[0]
    column_n = [None] * N + [df[feature][i - N] for i in range(N, rows)]
    column_name = "{}_{}".format(feature, N)
    df[column_name] = column_n
    
def make_predict_data(data,target):
    df = data.copy(deep=False)
    df = df.append(pd.Series([0,0,0],index = ['PM2.5','PM10','O3']),ignore_index = True)
    for N in range(1,49):
        feature_vector(df,target,N)
    df = df[48:].reset_index(drop=True)

    return(df)

In [30]:
sub1 = make_predict_data(data, "PM2.5")

In [31]:
sub1.iloc[len(data)-1:,]
feature = 'PM2.5'
features = ['PM2.5_'+str(i+1) for i in range(48)]
y = pd.DataFrame(sub1[feature]) #
X = pd.DataFrame(sub1[features]) # 48

In [32]:
y.shape

(544, 1)

In [33]:
X.shape

(544, 48)

In [34]:
model = Sequential()
model.add(Dense(32,input_dim=48,activation="relu"))
model.add(Dropout(0.3))
for i in range(2):
    model.add(Dense(32,activation="relu"))
    model.add(Dropout(0.3))
model.add(Dense(1))

In [35]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [36]:
hist = model.fit(X, y, epochs=200, batch_size=32)

Epoch 1/200
544/544 [==============================] - 0s 809us/step - loss: nan
Epoch 2/200
544/544 [==============================] - 0s 70us/step - loss: nan
Epoch 3/200
544/544 [==============================] - 0s 59us/step - loss: nan
Epoch 4/200
544/544 [==============================] - 0s 56us/step - loss: nan
Epoch 5/200
544/544 [==============================] - 0s 59us/step - loss: nan
Epoch 6/200
544/544 [==============================] - 0s 63us/step - loss: nan
Epoch 7/200
544/544 [==============================] - 0s 58us/step - loss: nan
Epoch 8/200
544/544 [==============================] - 0s 58us/step - loss: nan
Epoch 9/200
544/544 [==============================] - 0s 56us/step - loss: nan
Epoch 10/200
544/544 [==============================] - 0s 55us/step - loss: nan
Epoch 11/200
544/544 [==============================] - 0s 58us/step - loss: nan
Epoch 12/200
544/544 [==============================] - 0s 55us/step - loss: nan
Epoch 13/200
544/544 [==============

In [ ]:
def smape(actual, predicted):
    dividend= np.abs(np.array(actual) - np.array(predicted))
    denominator = np.array(actual) + np.array(predicted)
    
    return 2 * np.mean(np.divide(dividend, denominator, out=np.zeros_like(dividend), where=denominator!=0, casting='unsafe'))

In [1]:
dividend= np.abs(np.array(Y) - np.array(hypothesis))
denominator = np.array(Y) + np.array(hypothesis)

NameError: name 'np' is not defined

In [91]:
pred = model.predict(X)
pred

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
      

In [49]:
def train_model(data, feature):
    train_data = data
    test_data = data.iloc[len(data)-1:,]
    if feature == 'PM2.5':
        features = ['PM2.5_'+str(i+1) for i in range(48)]
    elif feature == 'PM10':
        features = ['PM10_'+str(i+1) for i in range(48)]
    elif feature == 'O3':
        features = ['O3_'+str(i+1) for i in range(48)]
    y = np.array(train_data[feature])
    X = np.array(train_data[features])
    clf = RandomForestRegressor(n_estimators = 10)
    clf.fit(X,y)
    return (clf)

In [5]:
# 1시간 단위로 생성하는 predict_generation\n",
def predict_generate(data,feature,clf):
    train_data = data
    test_data = data.iloc[len(data)-1:,]
    if feature == 'PM2.5':
        features = ['PM2.5_'+str(i+1) for i in range(48)]
    elif feature == 'PM10':
        features = ['PM10_'+str(i+1) for i in range(48)]
    elif feature == 'O3':
        features = ['O3_'+str(i+1) for i in range(48)]
    y = np.array(train_data[feature])
    X = np.array(train_data[features])
    prediction = clf.predict(test_data[features])

    return(prediction)

In [7]:
# predict_generation에 들어갈 data를 생성하는 make_predict_data\n",
